In [1]:
import pandas as pd
import numpy as np
import random
import time
import project
import founder
import requests
from bs4 import BeautifulSoup
from project import get_project, get_rewards
from founder import get_profile, get_bio
from featurizer1 import featurize
from pymongo import MongoClient
import urllib2

import multiprocessing
import sys
import threading
from timeit import Timer



In [40]:
# Initialize MongoDB database and collection
client = MongoClient()
db = client['ksdb']
collection = db['ksdata']

# Load array of project_id, founder_id and urls

df = pd.read_csv('id_url_list.csv', dtype=object, header=None)
id_url_list = df.values
sl = id_url_list[:7360]
sml = sl.copy()



In [3]:
def subsample(arr, p=1):
    '''
    Returns random subsample of 2D array. Default sample
    size is 100.
    '''
    mask = np.random.choice([True, False], arr.shape[0],
        p = [p, 1 - p])

    sub = arr[mask]

    return sub

def id_generator(arr):
    '''
    Create new generator.
    '''
    return (x for x in arr)

def extract(generator):
    '''
    Scrapes Kickstarter pages and parses features into
    MongoDB database. This function calls the featurize
    function from the featurizer module to insert data
    into the MongoDB database.
    '''
    go = True
    progress = 0
    skipped = 0
    failed = 0

    while go:
        block_size = random.randint(5, 10)
        wait_time = random.randint(2, 4)
        wait = False

        print '\n'
        print 'Block size: {0}'.format(block_size)

        for i in xrange(0, block_size):
            project_id,founder_id, project_url, founder_url, rewards_url = (x for x in generator.next())

            collection_check = set(db.ksdata.distinct('project_id', {}))
            if project_id in collection_check:
                    print "Already scraped"
                    skipped += 1
                    wait = False


            else:
                try:
                    project_soup, project_url, status1 = get_project(project_url)

                    founder_soup, founder_url, status2 = get_profile(founder_url)

                    rewards_soup, rewards_url, status3 = get_rewards(rewards_url)

                    if (status1 & status2 & status3) == 200:
                        featurize(project_id, founder_id, project_url, founder_url, rewards_url, project_soup, founder_soup, rewards_soup, collection)

                        progress += 1

                        wait = True

                except requests.ConnectionError:
                    failed +=1

        print '\n'

        print 'Scraped: {}'.format(progress)

        print 'Skipped: {}'.format(skipped)
        
        print 'Failed: {}'.format(failed)

        print 'Next block: {}s'.format(wait_time)

        if wait:
            time.sleep(wait_time)

        else:
            pass


In [4]:
# def get_rewards(url_rewards):
#     content = requests.get(url_rewards).content
#     soup = BeautifulSoup(content, 'html.parser')
#     r = requests.get(url_rewards)
#     status = r.status_code

#     print '({0})'.format(status), url_rewards

#     return soup, url_rewards, status

In [4]:
gen = id_generator(id_url_list)

In [ ]:
# extract(gen)

In [21]:
ls

__init__.py            founder.py             scraper.py
data_processing.ipynb  founder.pyc            scraper2.ipynb
data_processing.py     id_url_list.csv        subset_list.csv
export_mongo.py        project.py             url_list.csv
featurizer1.py         project.pyc
featurizer1.pyc        sample_ids.csv


# 

In [18]:
def scrape_parallel_concurrent(pool_size):
    """
    Uses multiple processes to make requests to scrape website.
    :param pool_size: number of worker processes
    """
    pool = multiprocessing.Pool(pool_size)

    for project in sml:
        pool.map(pool_extract, project)
        pool.close()
        pool.join()

In [ ]:
pool = multiprocessing.Pool(3)
pool.map(pool_extract, id_url_list)
pool.close()
pool.join()

/Users/lizsia/anaconda/lib/python2.7/site-packages/pymongo/topology.py:143: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#using-pymongo-with-multiprocessing>
  "MongoClient opened before fork. Create MongoClient "


Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Already scraped

Failed: 0

Scraped: 0
Skipped: 1
Failed: 0


/Users/lizsia/anaconda/lib/python2.7/site-packages/pymongo/topology.py:143: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#using-pymongo-with-multiprocessing>
  "MongoClient opened before fork. Create MongoClient "


Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Already scraped

Skipped: 1

Failed: 0
Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Already scraped


Already scraped
Scraped: 0
Already scraped
Skipped: 1

Failed: 0



/Users/lizsia/anaconda/lib/python2.7/site-packages/pymongo/topology.py:143: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#using-pymongo-with-multiprocessing>
  "MongoClient opened before fork. Create MongoClient "


Scraped: 0
Skipped: 1
Already scraped
Failed: 0


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped

Already scraped


Scraped: 0

Skipped: 1
Scraped: 0
Skipped: 1
Failed: 0
Failed: 0
Already scraped



Scraped: 0
Skipped: 1

Failed: 0
Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Already scraped

Skipped: 1

Failed: 0
Scraped: 0
Skipped: 1
Failed: 0
Already scraped

Already scraped

Scraped: 0

Skipped: 1

Failed: 0
Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Already scraped
Skipped: 1

Failed: 0

Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Scraped: 0
Already scraped
Skipped: 1
Failed: 0


Scraped: 0
Skipped: 1
Failed: 0
Already scraped


Already scraped
Scraped: 0

Skipped: 1

Scraped: 0
Failed: 0
Skipp

In [41]:
def pool_extract(gen):
    failed = 0
    skipped = 0
    progress = 0
    
    project_id,founder_id, project_url, founder_url, rewards_url = gen

    collection_check = set(db.ksdata.distinct('project_id', {}))
    if int(project_id) in collection_check:
            print "Already scraped"
            skipped += 1
    else:
        try:
            project_soup, project_url, status1 = get_project(project_url)

            founder_soup, founder_url, status2 = get_profile(founder_url)

            rewards_soup, rewards_url, status3 = get_rewards(rewards_url)

            if (status1 & status2 & status3) == 200:
                featurize(project_id, founder_id, project_url, founder_url, rewards_url, project_soup, founder_soup, rewards_soup, collection)

                progress += 1

                wait = True

        except requests.ConnectionError:
            failed +=1
    
    print '\n'

    print 'Scraped: {}'.format(progress)

    print 'Skipped: {}'.format(skipped)

    print 'Failed: {}'.format(failed)

